In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

In [13]:
store_train = pd.read_csv('https://raw.githubusercontent.com/PauloFS01/dataframes/main/store_sales_train.csv', parse_dates=['date'],
    infer_datetime_format=True)
store_test= pd.read_csv('https://raw.githubusercontent.com/PauloFS01/dataframes/main/store_sales_test.csv', parse_dates=['date'],
    infer_datetime_format=True)

store_train['date'] = store_train.date.dt.to_period('D')
store_train = store_train.set_index(['store_nbr', 'family', 'date']).sort_index()

store_test['date'] = store_test.date.dt.to_period('D')
store_test = store_test.set_index(['store_nbr', 'family', 'date']).sort_index()

/tmp/ipykernel_20497/3953057702.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  store_train = pd.read_csv('https://raw.githubusercontent.com/PauloFS01/dataframes/main/store_sales_train.csv', parse_dates=['date'],
/tmp/ipykernel_20497/3953057702.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  store_test= pd.read_csv('https://raw.githubusercontent.com/PauloFS01/dataframes/main/store_sales_test.csv', parse_dates=['date'],


In [ ]:
family_sales = (
    store_train
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

In [18]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)

def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)

## Analyze
First we will analyze dataset to understand our problem

In [14]:
print("Training Data", "\n" + "-" * 13 + "\n", store_train)
print("\n")
print("Test Data", "\n" + "-" * 9 + "\n", store_test)

Training Data 
-------------
                                  Unnamed: 0       id  sales  onpromotion
store_nbr family     date                                               
1         AUTOMOTIVE 2017-08-11     2991978  2991978    1.0            0
                     2017-08-12     2993760  2993760    6.0            0
                     2017-08-13     2995542  2995542    1.0            0
                     2017-08-14     2997324  2997324    1.0            0
                     2017-08-15     2999106  2999106    4.0            0
...                                     ...      ...    ...          ...
54        SEAFOOD    2017-08-11     2993627  2993627    0.0            0
                     2017-08-12     2995409  2995409    1.0            1
                     2017-08-13     2997191  2997191    2.0            0
                     2017-08-14     2998973  2998973    0.0            0
                     2017-08-15     3000755  3000755    3.0            0

[10000 rows x 4 colu

The training set ends on **2017-08-15**, which gives us the forecast origin. The test set comprises the dates **2017-08-16** to **2017-08-31**, and this gives us the forecast horizon. There is one step between the origin and horizon, so we have a lead time of one day.
Put another way, we need a **16-step** forecast with a 1-step lead time. We can use lags starting with **lag 1**, and we make the entire **16-step forecast** using features from **2017-08-15**.

In [22]:
y = family_sales.loc[:, 'sales']

# YOUR CODE HERE: Make 4 lag features
X =  make_lags(y, lags=4)
# YOUR CODE HERE: Make multistep target
y = make_multistep_target(y, steps=16)
y, X = y.align(X, join='inner', axis=0)

In [25]:
y.head()

y_step_1                                                          \
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-08-10   6.757576  0.060606  6.848485  3338.636364  0.000000   528.041394   
2017-08-11   8.166667  0.129630  6.500000  3502.055556  0.000000   522.305296   
2017-08-12   7.462963  0.055556  6.833333  3376.259259  0.000000   507.473111   
2017-08-13   8.907407  0.166667  8.018519  3747.296296  0.018519   615.976981   
2017-08-14   5.407407  0.166667  6.240741  3237.629630  0.000000   518.564019   

                                                             ... y_step_16  \
family     CELEBRATION     CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                         ...             
2017-08-10   20.515152   931.575758  761.878788  244.285909  ...       NaN   
2017-08-11   16.111111  1031.148148  831.648148  345.934963  ...       NaN   
2017-08-12   15.722222  1072.870370  793.592593  290.553574  ...       NaN   
2017-08-13   10.074074  1143.648148  928.000000  325.801371  ...       NaN   
2017-08-14   11.000000  1019.111111  797.222222  271.532241  ...       NaN   

                                                                             \
family     MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY   
date                                                                          
2017-08-10   NaN           NaN          NaN                     NaN     NaN   
2017-08-11   NaN           NaN          NaN                     NaN     NaN   
2017-08-12   NaN           NaN          NaN                     NaN     NaN   
2017-08-13   NaN           NaN          NaN                     NaN     NaN   
2017-08-14   NaN           NaN          NaN                     NaN     NaN   

                                                                      
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                  
2017-08-10            NaN     NaN                        NaN     NaN  
2017-08-11            NaN     NaN                        NaN     NaN  
2017-08-12            NaN     NaN                        NaN     NaN  
2017-08-13            NaN     NaN                        NaN     NaN  
2017-08-14            NaN     NaN                        NaN     NaN  

[5 rows x 528 columns]